In [61]:
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 30


In [62]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf   

# Creating the teams dataframe

In [63]:
TEAMNAMES=pd.read_csv('../Data/from_sqlite_Data/Team.csv')
LEAGUENAMES= pd.read_csv('../Data/from_sqlite_Data/League.csv')
df2 = pd.read_csv('../Data/from_sqlite_Data/bigfinal.csv')
df2.drop(['Unnamed: 0'],axis=1,inplace=True)
df2.drop(['league_id'],axis=1,inplace=True)
df2.drop(['id'],axis=1,inplace=True)

In [65]:
season = df2.iloc[:,:7]
FTR=[]
for i in range(len(season)):
    if season['FTHG'][i]>season['FTAG'][i]:
        FTR.append('H')
    if season['FTHG'][i]<season['FTAG'][i]:
        FTR.append('A')
    if season['FTHG'][i]==season['FTAG'][i]:
        FTR.append('D')
    
season['FTR']=FTR   

In [66]:
season=season.sort('Date')

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [67]:
season.reset_index(inplace=True)


In [68]:
season.drop(['index'],axis=1,inplace=True)

In [69]:
teams=season.HomeTeam.unique()
allteamsresults=[]
for i in teams:
    path=pd.DataFrame()            
    path = season[(season["HomeTeam"] == i) | (season['AwayTeam'] == i )] 
    allteamsresults.append(MyDf(path))

In [70]:
def point_in_round_k(ALL_TEAM,i,team_index,where):
    
    home=where
    away= not home
    
    if home:
        if ALL_TEAM[team_index].df['FTR'][i]=='H':
            return 3
        if ALL_TEAM[team_index].df['FTR'][i]=='D':
            return 1
        if ALL_TEAM[team_index].df['FTR'][i]=='A':
            return 0
    #   
    if away:
        if ALL_TEAM[team_index].df['FTR'][i]=='H':
            return 0
        if ALL_TEAM[team_index].df['FTR'][i]=='D':
            return 1
        if ALL_TEAM[team_index].df['FTR'][i]=='A':
            return 3

In [71]:
def zero_or_big(a,b):
    if a-b>0:
        return a-b
    return 0

In [72]:
def poin_in_last_n_mathces(ALL_TEAM,i,team_index,where,round_index,n):
    if round_index<=n:
        return point_in_round_k(ALL_TEAM,i,team_index,where)
        

## Allteamsresults is a list of dataframes and each dataframe represents a specific team. To access info on a team use team index. The position of the team in the list "teams" gives  the index of the team in the "allteamsresults" list of dataframes.

## Now, lets calculate some other features

## 1) Goal scored and received - home and away.
### We will shift of one match back each calculation, which means, each row will have the present match and the statistics goals scored up to that day (before the match).

In [73]:
def calculate_for_team_i(ALL_TEAM,team_lists,team_index):
    
    #create the lists for goals 
    Goal_scored_home=[]
    Goal_received_home=[]
    Goal_scored_away=[]
    Goal_received_away=[]
    
    #total_goal
    total_goal_scored=[]
    total_goal_received=[]
    
    #create the list for averages goals
    Av_Goal_scored_home=[]
    Av_Goal_received_home=[]
   

    Av_Goal_scored_away=[]
    Av_Goal_received_away=[]
       
    #create the list for averages total goals
    Av_total_goal_scored=[]
    Av_total_goal_received=[]
    
    #create the lists for points
    Point_home=[]
    Point_away=[]

    total_point=[]

    #create the list for average Points
    Av_Point_home=[]
    Av_Point_away=[]

    Av_total_point=[]
    
    
    #Point in the last mathces
    
    T_L3M_Pnt=[]
    T_L5M_Pnt=[]
    
    H_L3M_Pnt=[]
    A_L3M_Pnt=[]
    
    H_L5M_Pnt=[]
    A_L5M_Pnt=[]
    
    
    
    #ROUND_INDEX
    round_index=[]

    
    k=1
    
    Goal_scored_home.append(0)
    Goal_received_home.append(0)
    Goal_scored_away.append(0)
    Goal_received_away.append(0)
    total_goal_scored.append(0)
    total_goal_received.append(0)
    Av_Goal_scored_home.append(0)
    Av_Goal_received_home.append(0)
    Av_Goal_scored_away.append(0)
    Av_Goal_received_away.append(0)
    Av_total_goal_scored.append(0)
    Av_total_goal_received.append(0)
    Point_home.append(0)
    Point_away.append(0)
    total_point.append(0)
    Av_Point_home.append(0)
    Av_Point_away.append(0)
    Av_total_point.append(0) 
    T_L3M_Pnt.append(0)
    T_L5M_Pnt.append(0)
    #H_L3M_Pnt.append(0)
    #A_L3M_Pnt.append(0)
    #H_L5M_Pnt.append(0)
    #A_L5M_Pnt.append(0)
    
    
    for h in allteamsresults[team_index].df['season'].unique():
    
        seasonal_index=allteamsresults[team_index].df[allteamsresults[team_index].df['season']==h].index
        Home=0
        Away=0
        total=0
        round_in=1
        
        for i in seasonal_index:
            
            round_index.append(round_in)
            round_index_2=round_in-1
            
            home_1=ALL_TEAM[team_index].df['HomeTeam'][i]==team_lists[team_index]
            away_1= not home_1

            if (round_in<=3):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(3)
                        #H_L3M_Pnt.append(3)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1)                        
                        #H_L3M_Pnt.append(1)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(0)                        
                        #H_L3M_Pnt.append(0)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                #   
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(0)
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(0)
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1)
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(1)
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(3)
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(3)
            #                        
            if (round_in<=5):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(3)
                        #H_L5M_Pnt.append(3)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1)
                        #H_L5M_Pnt.append(1)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(0)
                        #H_L5M_Pnt.append(0)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(0)
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(0)

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1)
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(1)

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(3)
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(3)
                        
            '''if (round_in==3):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(3++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(3)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])                        
                        #H_L3M_Pnt.append(1)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(0++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])                        
                        #H_L3M_Pnt.append(0)
                        #A_L3M_Pnt.append(A_L3M_Pnt[k-1])
                #   
                #
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(0++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(0)
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L3M_Pnt.append(1+T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(1)
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(3++T_L3M_Pnt[round_index_2-1]+T_L3M_Pnt[round_index_2-2])
                        #H_L3M_Pnt.append(H_L3M_Pnt[k-1])
                        #A_L3M_Pnt.append(3)
            
            if (round_in==5):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(3+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(3)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(1)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(0+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(0)
                        #A_L5M_Pnt.append(A_L5M_Pnt[k-1])
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(0+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(0)

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(1)

                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(3+T_L5M_Pnt[round_index_2-1]+T_L5M_Pnt[round_index_2-2]+T_L5M_Pnt[round_index_2-3]
                                        +T_L5M_Pnt[round_index_2-4])
                        #H_L5M_Pnt.append(H_L5M_Pnt[k-1])
                        #A_L5M_Pnt.append(3)
'''
            if (round_in>3):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(3+
                                         zero_or_big(T_L3M_Pnt[round_index_2-1],T_L3M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L3M_Pnt[round_index_2-2],T_L3M_Pnt[round_index_2-3]))

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':  
                        T_L3M_Pnt.append(1+
                                         zero_or_big(T_L3M_Pnt[round_index_2-1],T_L3M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L3M_Pnt[round_index_2-2],T_L3M_Pnt[round_index_2-3]))
                    
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(0+
                                         zero_or_big(T_L3M_Pnt[round_index_2-1],T_L3M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L3M_Pnt[round_index_2-2],T_L3M_Pnt[round_index_2-3]))
                        
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L3M_Pnt.append(0+
                                         zero_or_big(T_L3M_Pnt[round_index_2-1],T_L3M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L3M_Pnt[round_index_2-2],T_L3M_Pnt[round_index_2-3]))

                    if ALL_TEAM[team_index].df['FTR'][i]=='D':  
                        T_L3M_Pnt.append(1+
                                         zero_or_big(T_L3M_Pnt[round_index_2-1],T_L3M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L3M_Pnt[round_index_2-2],T_L3M_Pnt[round_index_2-3]))
                    
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L3M_Pnt.append(3+
                                        zero_or_big(T_L3M_Pnt[round_index_2-1],T_L3M_Pnt[round_index_2-2])+
                                        zero_or_big(T_L3M_Pnt[round_index_2-2],T_L3M_Pnt[round_index_2-3]))

            if (round_in>5):
                if home_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(3+
                                         zero_or_big(T_L5M_Pnt[round_index_2-1],T_L5M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-2],T_L5M_Pnt[round_index_2-3])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-3],T_L5M_Pnt[round_index_2-4])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-4],T_L5M_Pnt[round_index_2-5]))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1+
                                         zero_or_big(T_L5M_Pnt[round_index_2-1],T_L5M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-2],T_L5M_Pnt[round_index_2-3])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-3],T_L5M_Pnt[round_index_2-4])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-4],T_L5M_Pnt[round_index_2-5]))
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(0+
                                         zero_or_big(T_L5M_Pnt[round_index_2-1],T_L5M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-2],T_L5M_Pnt[round_index_2-3])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-3],T_L5M_Pnt[round_index_2-4])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-4],T_L5M_Pnt[round_index_2-5]))
                        
                if away_1:
                    if ALL_TEAM[team_index].df['FTR'][i]=='H':
                        T_L5M_Pnt.append(0+
                                         zero_or_big(T_L5M_Pnt[round_index_2-1],T_L5M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-2],T_L5M_Pnt[round_index_2-3])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-3],T_L5M_Pnt[round_index_2-4])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-4],T_L5M_Pnt[round_index_2-5]))
                        
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='D':
                        T_L5M_Pnt.append(1+
                                        zero_or_big(T_L5M_Pnt[round_index_2-1],T_L5M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-2],T_L5M_Pnt[round_index_2-3])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-3],T_L5M_Pnt[round_index_2-4])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-4],T_L5M_Pnt[round_index_2-5]))
                        
                        
                    if ALL_TEAM[team_index].df['FTR'][i]=='A':
                        T_L5M_Pnt.append(3+
                                         zero_or_big(T_L5M_Pnt[round_index_2-1],T_L5M_Pnt[round_index_2-2])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-2],T_L5M_Pnt[round_index_2-3])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-3],T_L5M_Pnt[round_index_2-4])+
                                         zero_or_big(T_L5M_Pnt[round_index_2-4],T_L5M_Pnt[round_index_2-5]))
                        
            
            round_in=round_in+1
            

            #HOME
            if ALL_TEAM[team_index].df['HomeTeam'][i]==team_lists[team_index]:

                #Goal scored, received and total
                Goal_scored_home.append(ALL_TEAM[team_index].df['FTHG'][i]+Goal_scored_home[k-1])
                Goal_received_home.append(ALL_TEAM[team_index].df['FTAG'][i]+Goal_received_home[k-1])  
                Goal_scored_away.append(Goal_scored_away[k-1])
                Goal_received_away.append(Goal_received_away[k-1])
                total_goal_scored.append(total_goal_scored[k-1]+ALL_TEAM[team_index].df['FTHG'][i])
                total_goal_received.append(total_goal_received[k-1]+ALL_TEAM[team_index].df['FTAG'][i])

                #Goal average
                Home=Home+1
                total=Home+Away
                Av_Goal_scored_home.append(float(Goal_scored_home[k])/Home)
                Av_Goal_received_home.append(float(Goal_received_home[k])/Home)
                Av_Goal_scored_away.append(Av_Goal_scored_away[k-1])
                Av_Goal_received_away.append(Av_Goal_received_away[k-1])
                Av_total_goal_scored.append(float(total_goal_scored[k])/total)
                Av_total_goal_received.append(float(total_goal_received[k])/total)


                #Point winning
                if ALL_TEAM[team_index].df['FTR'][i]=='H':
                    Point_home.append(3+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)
                    total_point.append(3+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point draw
                if ALL_TEAM[team_index].df['FTR'][i]=='D':
                    Point_home.append(1+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)
                    total_point.append(1+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point lost
                if ALL_TEAM[team_index].df['FTR'][i]=='A':
                    Point_home.append(0+Point_home[k-1])
                    Av_Point_home.append(float(Point_home[k])/Home)   
                    total_point.append(0+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                Point_away.append(Point_away[k-1])
                Av_Point_away.append(Av_Point_away[k-1])

            #AWAY
            if ALL_TEAM[team_index].df['HomeTeam'][i]!=team_lists[team_index]:

                #Goal scored, received and total
                Goal_scored_away.append(ALL_TEAM[team_index].df['FTAG'][i]+Goal_scored_away[k-1])
                Goal_received_away.append(ALL_TEAM[team_index].df['FTHG'][i]+Goal_received_away[k-1])
                Goal_scored_home.append(Goal_scored_home[k-1])
                Goal_received_home.append(Goal_received_home[k-1])
                total_goal_scored.append(total_goal_scored[k-1]+ALL_TEAM[team_index].df['FTAG'][i])
                total_goal_received.append(total_goal_received[k-1]+ALL_TEAM[team_index].df['FTHG'][i])

                #Goal average
                Away=Away+1
                total=Home+Away

                Av_Goal_scored_away.append(float(Goal_scored_away[k])/Away)
                Av_Goal_received_away.append(float(Goal_received_away[k])/Away)
                Av_Goal_scored_home.append(Av_Goal_scored_home[k-1])
                Av_Goal_received_home.append(Av_Goal_received_home[k-1])
                Av_total_goal_scored.append(float(total_goal_scored[k])/total)
                Av_total_goal_received.append(float(total_goal_received[k])/total)


                #Point winning
                if ALL_TEAM[team_index].df['FTR'][i]=='A':
                    Point_away.append(3+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)
                    total_point.append(3+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point draw
                if ALL_TEAM[team_index].df['FTR'][i]=='D':
                    Point_away.append(1+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)

                    total_point.append(1+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)

                #Point lost
                if ALL_TEAM[team_index].df['FTR'][i]=='H':
                    Point_away.append(0+Point_away[k-1])
                    Av_Point_away.append(float(Point_away[k])/Away)

                    total_point.append(0+total_point[k-1])
                    Av_total_point.append(float(total_point[k])/total)


                Point_home.append(Point_home[k-1])
                Av_Point_home.append(Av_Point_home[k-1])
                
                


            k=k+1

        
        Goal_scored_home[-1]=0
        Goal_received_home[-1]=0
        Goal_scored_away[-1]=0
        Goal_received_away[-1]=0
        total_goal_scored[-1]=0
        total_goal_received[-1]=0


        Av_Goal_scored_home[-1]=0
        Av_Goal_received_home[-1]=0
        Av_Goal_scored_away[-1]=0
        Av_Goal_received_away[-1]=0
        Av_total_goal_scored[-1]=0
        Av_total_goal_received[-1]=0


        Point_home[-1]=0
        Point_away[-1]=0
        Av_Point_home[-1]=0
        Av_Point_away[-1]=0

        total_point[-1]=0
        Av_total_point[-1]=0
        
        
        T_L3M_Pnt[-1]=0
        T_L5M_Pnt[-1]=0
        

    del Goal_scored_home[-1]
    del Goal_received_home[-1]
    del Goal_scored_away[-1]
    del Goal_received_away[-1]
    del total_goal_scored[-1]
    del total_goal_received[-1]


    del Av_Goal_scored_home[-1]
    del Av_Goal_received_home[-1]
    del Av_Goal_scored_away[-1]
    del Av_Goal_received_away[-1]
    del Av_total_goal_scored[-1]
    del Av_total_goal_received[-1]


    del Point_home[-1]
    del Point_away[-1]
    del Av_Point_home[-1]
    del Av_Point_away[-1]

    del total_point[-1]
    del Av_total_point[-1]
    
    del T_L3M_Pnt[-1]
    del T_L5M_Pnt[-1]
    
    ALL_TEAM[team_index].df['Round']=round_index
    
    ALL_TEAM[team_index].df['T_L3M']=T_L3M_Pnt
    ALL_TEAM[team_index].df['T_L5M']=T_L5M_Pnt
    
    ALL_TEAM[team_index].df['GS_H']=Goal_scored_home
    ALL_TEAM[team_index].df['GS_A']=Goal_scored_away
    ALL_TEAM[team_index].df['GR_H']=Goal_received_home
    ALL_TEAM[team_index].df['GR_A']=Goal_received_away
    ALL_TEAM[team_index].df['TGS']=total_goal_scored
    ALL_TEAM[team_index].df['TGR']=total_goal_received
    
    
    ALL_TEAM[team_index].df['AGS_H']=Av_Goal_scored_home
    ALL_TEAM[team_index].df['AGS_A']=Av_Goal_scored_away
    ALL_TEAM[team_index].df['AGR_H']=Av_Goal_received_home
    ALL_TEAM[team_index].df['AGR_A']=Av_Goal_received_away
    ALL_TEAM[team_index].df['ATGS']=Av_total_goal_scored
    ALL_TEAM[team_index].df['ATGR']=Av_total_goal_received

    
    ALL_TEAM[team_index].df['Pnt_H']=Point_home
    ALL_TEAM[team_index].df['Pnt_A']=Point_away
    ALL_TEAM[team_index].df['TOTPT']=total_point

    
    ALL_TEAM[team_index].df['Av_Pnt_H']=Av_Point_home
    ALL_TEAM[team_index].df['Av_pnt_A']=Av_Point_away
    ALL_TEAM[team_index].df['Av_tot_pnt']=Av_total_point
    
    
    return ALL_TEAM[team_index].df


In [74]:
calculate_for_team_i(allteamsresults,teams,1)

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:471: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:473: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,country_id,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Round,T_L3M,T_L5M,GS_H,GS_A,GR_H,GR_A,TGS,TGR,AGS_H,AGS_A,AGR_H,AGR_A,ATGS,ATGR,Pnt_H,Pnt_A,TOTPT,Av_Pnt_H,Av_pnt_A,Av_tot_pnt
1,Switzerland Super League,2008/2009,2008-07-19 00:00:00,FC Aarau,FC Sion,3,1,H,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0.000000,0.000000,0.000000
8,Switzerland Super League,2008/2009,2008-07-24 00:00:00,FC Vaduz,FC Aarau,0,2,A,2,3,3,3,0,1,0,3,1,3.000000,0.000000,1.000000,0.000000,3.000000,1.000000,3,0,3,3.000000,0.000000,3.000000
11,Switzerland Super League,2008/2009,2008-07-27 00:00:00,FC Aarau,FC Zürich,2,1,H,3,3,3,3,2,1,0,5,1,3.000000,2.000000,1.000000,0.000000,2.500000,0.500000,3,3,6,3.000000,3.000000,3.000000
15,Switzerland Super League,2008/2009,2008-08-02 00:00:00,Neuchâtel Xamax,FC Aarau,0,0,D,4,3,3,5,2,2,0,7,2,2.500000,2.000000,1.000000,0.000000,2.333333,0.666667,6,3,9,3.000000,3.000000,3.000000
19,Switzerland Super League,2008/2009,2008-08-06 00:00:00,AC Bellinzona,FC Aarau,1,1,D,5,4,1,5,2,2,0,7,2,2.500000,1.000000,1.000000,0.000000,1.750000,0.500000,6,4,10,3.000000,2.000000,2.500000
35,Switzerland Super League,2008/2009,2008-08-09 00:00:00,FC Aarau,FC Luzern,1,0,H,6,1,1,5,3,2,1,8,3,2.500000,1.000000,1.000000,0.333333,1.600000,0.600000,6,5,11,3.000000,1.666667,2.200000
161,Switzerland Super League,2008/2009,2008-08-24 00:00:00,BSC Young Boys,FC Aarau,3,3,D,7,4,6,6,3,2,1,9,3,2.000000,1.000000,0.666667,0.333333,1.500000,0.500000,9,5,14,3.000000,1.666667,2.333333
185,Switzerland Super League,2008/2009,2008-08-30 00:00:00,FC Aarau,FC Basel,0,2,A,8,2,1,6,6,2,4,12,6,2.000000,1.500000,0.666667,1.000000,1.714286,0.857143,9,6,15,3.000000,1.500000,2.142857
275,Switzerland Super League,2008/2009,2008-09-13 00:00:00,Grasshopper Club Zürich,FC Aarau,0,0,D,9,3,5,6,6,4,4,12,8,1.500000,1.500000,1.000000,1.000000,1.500000,1.000000,9,6,15,2.250000,1.500000,1.875000
466,Switzerland Super League,2008/2009,2008-09-27 00:00:00,FC Sion,FC Aarau,1,1,D,10,4,6,6,6,4,4,12,8,1.500000,1.200000,1.000000,0.800000,1.333333,0.888889,9,7,16,2.250000,1.400000,1.777778


In [75]:
for i in range(len(teams)):
    allteamsresults[i].df=calculate_for_team_i(allteamsresults,teams,i)


/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:471: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:473: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:474: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [76]:
tmp=pd.DataFrame(columns=(
                          'H_ROUND',
    
                          'H_TP3',
                          'H_TP5',
                            
    
                          'H_GS_H', 
                          'H_GS_A', 
                          'H_GR_H', 
                          'H_GR_A', 
                          'H_TGS',
                          'H_TGR', 
                          
                          'H_AvGS_H' ,
                          'H_AvGS_A' ,
                          'H_AvGR_H',
                          'H_AvGR_A',
                          'H_Av_TGS',
                          'H_Av_TGR', 
                          
                          
                          'H_Pnt_H' ,
                          'H_Pnt_A',
                          'H_T_pnt',
                          'H_Av_Pnt_H',
                          'H_Av_pnt_A',
                          'H_Av_T_pnt',
                          
                          'A_ROUND',
    
                          'A_TP3',
                          'A_TP5',

                          'Aw_GS_H', 
                          'Aw_GS_A', 
                          'Aw_GR_H', 
                          'Aw_GR_A', 
                          'Aw_TGS',
                          'Aw_TGR', 
                          
                          'Aw_AvGS_H' ,
                          'Aw_AvGS_A' ,
                          'Aw_AvGR_H',
                          'Aw_AvGR_A',
                          'Aw_Av_TGS',
                          'Aw_Av_TGR', 
                          
                          
                          'Aw_Pnt_H' ,
                          'Aw_Pnt_A',
                          'Aw_T_pnt',
                          'Aw_Av_Pnt_H',
                          'Aw_Av_pnt_A',
                         
                          'Aw_Av_T_pnt',
                          
                          
                          
                         )
                )



In [77]:
for i in season.index:
    date=season['Date'][i]
    Home=season['HomeTeam'][i]
    Away=season['AwayTeam'][i]
    
    home_index=pd.Series(teams)[teams==Home].index[0]
    away_index=pd.Series(teams)[teams==Away].index[0]
    tobeappend=list(allteamsresults[home_index].df.loc[i].iloc[8:]) + list(allteamsresults[away_index].df.loc[i].iloc[8:])
    tmp.loc[i]=tobeappend

In [78]:
final=pd.merge(left=season, right=tmp, left_index=True, right_index=True)

In [93]:
final['DIFF_av_TP']=final['H_Av_T_pnt']-final['Aw_Av_T_pnt']
final['DIFF_av_HP-AP']=final['H_Av_Pnt_H']-final['Aw_Av_pnt_A']

final['SUMM_av_HGSH+AGRA']=final['H_AvGS_H']+final['Aw_AvGR_A']
final['SUMM_av_HGRH+AGSA']=final['H_AvGR_H']+final['Aw_AvGS_A']

final['SUMM_av_HTGS+ATGR']=final['H_Av_TGS']+final['Aw_Av_TGR']
final['SUMM_av_HTGR+ATGS']=final['H_Av_TGR']+final['Aw_Av_TGS']

final['Goal/No_Goal'] = ((final['FTHG']>0 ) * 1)*((final['FTAG']>0) * 1)
final['Over2.5/Under2.5']= ((final['FTHG'] + final['FTAG']) >2)*1


In [95]:
final.to_csv('../Data/from_sqlite_Data/realfinal.csv')

In [69]:
final=pd.read_csv('../Data/from_sqlite_Data/realfinal.csv')

In [70]:
final.drop(['Unnamed: 0'], axis=1,inplace=True)